In [1]:
from math import (atan2, pi, sqrt)    

from collections import OrderedDict

import os
import numpy as np
import torch
import torch.nn.functional as F

torch.set_printoptions(sci_mode=False)

from src.polar_traversal import (
    Bridge,
    make_latitudes,
    get_start,
    gather_path,
    cumulative_distances,
)

import src.polar_traversal as polar_traversal


def get_dist(x, y):
    return sqrt((x-0.5) **2 + (y-0.5) ** 2)

def get_angle(x, y, square_no):
    side = (square_no + 1) * 2
    corr = (side -1) / 2    
    angle = atan2(y -corr, x -corr)
    return angle if angle >= 0 else angle + 2 * pi


def get_sample(pos, normed, path_points, path_normals):    
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)

def single_path_sample(path, layer_no, total_layers):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    pos = (layer_no+1) / (total_layers+1)
    pnt, nrm, _ = get_sample(pos, normed, 
                             path['points'], path['normals'])
    return pnt, nrm

def get_angles_series(n):
    return [x for x in range(0, n*2+1, 2)]


def get_sample(pos, normed, path_points, path_normals):    
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)

def single_path_sample(path, layer_no, total_layers):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    pos = layer_no / total_layers    
    pnt, nrm, _ = get_sample(pos, normed, 
                             path['points'], path['normals'])
    return torch.tensor(pnt).float(), torch.tensor(nrm).float()

def square_indices(n):
    max_idx = 2 * (n+1)
    res = [[0, x] for x in range(max_idx -1, -1, -1)]\
        + [[x, 0] for x in range(1, max_idx)]\
        + [[max_idx -1, x] for x in range(1, max_idx)]\
        + [[x, max_idx -1] for x in range(max_idx -2, 0, -1)]
    return res

def shifted_square_indices(sq_no, square_no):    
    sq_ids =  square_indices(sq_no)
    ost = square_no - sq_no
    return [(i+ost, k+ost) for (i, k) in iter(sq_ids)]

def get_latitudes_angles(n):
    return (torch.arange(0, n) * (2 * pi / n)).tolist()

class Blueprint:
    
    def __init__(self, path, latitudes_num):
        self.path = path
        self.latitudes_num = latitudes_num
        self.bridge = Bridge(path)
        paths_path =  f'./data/paths_{latitudes_num}.pth'        
        if os.path.exists(paths_path):
            self.paths = torch.load(paths_path) 
        else:
            self.paths = self.gather_paths(latitudes_num)
        
    
    def gather_paths(self, latitudes_num=None):
        latitudes_num = latitudes_num or self.latitudes_num  
        latitude_angles = get_latitudes_angles(latitudes_num)        
        latitudes = make_latitudes(latitudes_num)
        bridge = self.bridge
        face_id, point, normal = get_start(bridge.mesh)
        paths = OrderedDict([])
        for i, (latitude, latitude_angle) in enumerate(zip(latitudes, latitude_angles)):
            print(i, latitudes_num)
            path = gather_path(latitude, face_id, point, normal, bridge)
            paths[latitude_angle] = path
        return paths
        
    def __repr__(self):
        return f'Blueprint: {self.path} latitudes_num: {self.latitudes_num}' 

    def get_border_paths(self, angle):        
        assert angle >=0 and angle < 2 * pi
        paths = self.paths
        angle_keys = list(paths.keys())
        for i, angle_key in enumerate(angle_keys):        
            if angle < angle_key:            
                return (paths[angle_keys[i % len(angle_keys)]],
                        paths[angle_keys[(i+1) % len(angle_keys)]])
        return (paths[angle_keys[0]], paths[angle_keys[-1]])

    def grid_sample(self, square_no, save_path=None):
        side = (square_no + 1) * 2
        res = self.load_result(save_path) \
            if save_path is not None and os.path.exists(save_path) \
            else self.create_result(side)
        
        face_id, point, normal = get_start(self.bridge.mesh)
        for sq_no in range(square_no+1):
            print(sq_no, res['sq_no'][0])
            if sq_no > res['sq_no'][0]:                                
                side = (sq_no + 1) * 2
                sq_ids = shifted_square_indices(sq_no, square_no)            
                angles_no = len(sq_ids)                          
                for (r, c) in sq_ids:
                    angle = get_angle(r, c, square_no)                    
                    path0, path1 = self.get_border_paths(angle)
                    pnt0, nrm0 = single_path_sample(path0, sq_no, square_no)
                    pnt1, nrm1 = single_path_sample(path1, sq_no, square_no)
                    pnt = (pnt0 + pnt1)/2
                    nrm = F.normalize((nrm0 + nrm1)/2, dim=0)
                    res['points'][0, :, c, r] = pnt
                    res['normals'][0, :, c, r] = nrm                
                    res['grid'][c, r] = float(sq_no)                
                res['sq_no'][0] = sq_no
                self.save_result(res, save_path)
        return res            
        
    def create_result(self, side):
        return {
            'points': torch.zeros(1, 3, side, side),
            'normals': torch.zeros(1, 3, side, side),
            'grid': torch.zeros(side, side) - 9,
            'sq_no': torch.tensor([-1]),
        }
    
    def load_result(self, path):
        loaded = np.load(path)
        
        return {
            'points': torch.tensor(loaded['points']).float(),
            'normals': torch.tensor(loaded['normals']).float(),
            'grid': torch.tensor(loaded['grid']).float(),
            'sq_no': torch.tensor(loaded['sq_no']).long(),
        }
    
    def save_result(self, res, path=None):
        #assert type('s')  == 'str'
        if path is not None:            
            print('path', path, res['sq_no'])
            np.savez_compressed(
                path,
                points=res['points'].numpy(),
                normals=res['normals'].numpy(),
                grid=res['grid'].numpy(),
                sq_no=res['sq_no'].numpy(),
            )
        

stl_path = './data/centered_3.stl'

square_no = 127
blueprint = Blueprint(stl_path, 4096)
print(blueprint)
#torch.save(blueprint.paths, f'./data/paths_{len(blueprint.paths)}.pth')
#blueprint_path = './data/blueprint_radial{}.npz'.format(square_no)
# print(blueprint_path)
# res = blueprint.grid_sample(square_no, blueprint_path)
# res['points'].shape, res['normals'].shape

Blueprint: ./data/centered_3.stl latitudes_num: 4096


In [6]:
square_no = 1
blueprint_path = './data/blueprint_radial_{}.npz'.format((square_no + 1) * 2)
print(blueprint_path)
res = blueprint.grid_sample(square_no, blueprint_path)
res['points'].shape, res['normals'].shape

./data/blueprint_radial_4.npz
0 tensor(-1)
path ./data/blueprint_radial_4.npz tensor([0])
1 tensor(0)
path ./data/blueprint_radial_4.npz tensor([1])


(torch.Size([1, 3, 4, 4]), torch.Size([1, 3, 4, 4]))

In [5]:
square_no = 1
side = (square_no + 1) * 2
side

4

In [65]:
square_no = 1
side = (square_no + 1) * 2
side

4

In [66]:
square_no = 2
side = (square_no + 1) * 2
side

6

In [68]:
# 01
# 1
0.5
# 0123
# 1
# 2
# 3
1.5
# 012345
2.5
def get_angle(x, y, square_no):
    side = (square_no + 1) * 2
    corr = (side -1) / 2    
    angle = atan2(y -corr, x -corr)
    return angle if angle >= 0 else angle + 2 * pi

get_angle(0, 0, 0)

0.5


3.9269908169872414

In [57]:
res['points']

tensor([[[[-0.3801,  0.3780,  0.4632,  0.4711],
          [-0.4931,  0.0052,  0.0051,  0.4797],
          [-0.2889,  0.0052,  0.0052,  0.4984],
          [-0.1947,  0.1769,  0.4273,  0.4932]],

         [[-0.3676, -0.3707, -0.1485, -0.0932],
          [ 0.4815, -0.0056, -0.0056,  0.0954],
          [ 0.8876, -0.0057, -0.0056,  0.2914],
          [ 0.9380,  0.9446,  0.7485,  0.4802]],

         [[ 0.0074,  0.0077,  0.0095,  0.0099],
          [ 0.0096,  0.7966,  0.7966,  0.0090],
          [ 0.0094,  0.7967,  0.7966,  0.0106],
          [ 0.0087,  0.0107,  0.0198,  0.0090]]]])

In [15]:
sq_no, square_no = 0, 1
sq_ids = shifted_square_indices(sq_no, square_no)   
sq_ids

[(1, 2), (1, 1), (2, 1), (2, 2)]

In [17]:
angle = get_angle(1, 2)
angle

1.2490457723982544

In [19]:
for lat_angle in  blueprint.paths.keys():
    print(lat_angle)

0.0
0.0015339808305725455
0.003067961661145091
0.004601942375302315
0.006135923322290182
0.0076699042692780495
0.00920388475060463
0.010737866163253784
0.012271846644580364
0.013805827125906944
0.015339808538556099
0.016873789951205254
0.01840776950120926
0.019941750913858414
0.02147573232650757
0.023009711876511574
0.02454369328916073
0.026077674701809883
0.02761165425181389
0.029145635664463043
0.030679617077112198
0.03221359848976135
0.03374757990241051
0.03528155758976936
0.03681553900241852
0.03834952041506767
0.03988350182771683
0.04141748324036598
0.04295146465301514
0.04448544234037399
0.04601942375302315
0.0475534051656723
0.04908738657832146
0.05062136799097061
0.052155349403619766
0.05368933081626892
0.05522330850362778
0.05675728991627693
0.058291271328926086
0.05982525274157524
0.061359234154224396
0.06289321184158325
0.0644271969795227
0.06596117466688156
0.06749515980482101
0.06902913749217987
0.07056311517953873
0.07209710031747818
0.07363107800483704
0.0751650631427764

1.3713788986206055
1.372912883758545
1.3744468688964844
1.3759808540344238
1.3775148391723633
1.3790488243103027
1.3805826902389526
1.382116675376892
1.3836506605148315
1.385184645652771
1.3867186307907104
1.38825261592865
1.3897866010665894
1.3913205862045288
1.3928545713424683
1.3943885564804077
1.3959225416183472
1.3974565267562866
1.398990511894226
1.4005244970321655
1.402058482170105
1.4035924673080444
1.4051264524459839
1.4066604375839233
1.4081944227218628
1.4097284078598022
1.4112623929977417
1.4127963781356812
1.4143303632736206
1.41586434841156
1.4173983335494995
1.418932318687439
1.4204663038253784
1.4220002889633179
1.4235341548919678
1.4250681400299072
1.4266021251678467
1.4281361103057861
1.4296700954437256
1.431204080581665
1.4327380657196045
1.434272050857544
1.4358060359954834
1.4373400211334229
1.4388740062713623
1.4404079914093018
1.4419419765472412
1.4434759616851807
1.4450099468231201
1.4465439319610596
1.448077917098999
1.4496119022369385
1.451145887374878
1.45267

2.6599228382110596
2.661456823348999
2.6629908084869385
2.664524793624878
2.6660587787628174
2.667592763900757
2.6691267490386963
2.6706607341766357
2.672194719314575
2.6737287044525146
2.675262451171875
2.6767964363098145
2.678330421447754
2.6798644065856934
2.681398391723633
2.6829323768615723
2.6844663619995117
2.686000347137451
2.6875343322753906
2.68906831741333
2.6906023025512695
2.692136287689209
2.6936702728271484
2.695204257965088
2.6967382431030273
2.698272228240967
2.6998062133789062
2.7013401985168457
2.702874183654785
2.7044081687927246
2.705942153930664
2.7074761390686035
2.709010124206543
2.7105441093444824
2.712078094482422
2.7136120796203613
2.715146064758301
2.7166800498962402
2.7182140350341797
2.719748020172119
2.7212820053100586
2.722815990447998
2.7243499755859375
2.725883960723877
2.7274179458618164
2.728951930999756
2.7304859161376953
2.7320199012756348
2.733553886413574
2.7350878715515137
2.736621856689453
2.7381558418273926
2.739689826965332
2.7412238121032715

4.098796844482422
4.100330829620361
4.101864814758301
4.10339879989624
4.10493278503418
4.106466770172119
4.108000755310059
4.109534740447998
4.1110687255859375
4.112602710723877
4.114136695861816
4.115670680999756
4.117204666137695
4.118738651275635
4.120272636413574
4.121806621551514
4.123340606689453
4.124874591827393
4.126408576965332
4.1279425621032715
4.129476547241211
4.13101053237915
4.13254451751709
4.134078502655029
4.135612487792969
4.137146472930908
4.138680458068848
4.140214443206787
4.141748428344727
4.143282413482666
4.1448163986206055
4.146350383758545
4.147884368896484
4.149418354034424
4.150952339172363
4.152486324310303
4.154020309448242
4.155554294586182
4.157088279724121
4.1586222648620605
4.16015625
4.161689758300781
4.163223743438721
4.16475772857666
4.1662917137146
4.167825698852539
4.1693596839904785
4.170893669128418
4.172427654266357
4.173961639404297
4.175495624542236
4.177029609680176
4.178563594818115
4.180097579956055
4.181631565093994
4.183165550231934
4

5.5253987312316895
5.526932716369629
5.528466701507568
5.530000686645508
5.531534671783447
5.533068656921387
5.534602642059326
5.536136627197266
5.537670612335205
5.5392045974731445
5.540738582611084
5.542272567749023
5.543806552886963
5.545340538024902
5.546874523162842
5.548408508300781
5.549942493438721
5.55147647857666
5.5530104637146
5.554544448852539
5.5560784339904785
5.557612419128418
5.559146404266357
5.560680389404297
5.562214374542236
5.563748359680176
5.565282344818115
5.566816329956055
5.568350315093994
5.569884300231934
5.571418285369873
5.5729522705078125
5.574486255645752
5.576020240783691
5.577554225921631
5.57908821105957
5.58062219619751
5.582156181335449
5.583690166473389
5.585224151611328
5.586758136749268
5.588292121887207
5.5898261070251465
5.591360092163086
5.592894077301025
5.594428062438965
5.595962047576904
5.597496032714844
5.599030017852783
5.600564002990723
5.602097988128662
5.603631973266602
5.605165958404541
5.6066999435424805
5.60823392868042
5.60976791

In [21]:
keys = list(blueprint.paths.keys())
len(keys)

4096

In [23]:
len(keys[:-1])

4095

In [25]:
4098 % 4096

2

In [39]:
def get_border_paths(angle, paths):
    assert angle >=0 and angle < 2 * pi
    angle_keys = list(paths.keys())
    for i, angle_key in enumerate(angle_keys):        
        if angle < angle_key:            
            return (paths[angle_keys[i % len(keys)]],
                    paths[angle_keys[(i+1) % len(keys)]])
    return (paths[angle_keys[0]], paths[angle_keys[-1]])

angle = get_angle(1, 2)
paths = blueprint.paths
path0, path1 = get_border_paths(angle, paths)
path0, path1

({'points': array([[ 0.00000000e+00,  0.00000000e+00,  7.93512133e-01],
         [ 5.23179260e-03, -5.70234526e-03,  7.96667119e-01],
         [ 5.06369385e-03,  4.23459819e-04,  7.92992290e-01],
         [ 3.59459009e-03,  2.50395235e-03,  7.91804650e-01],
         [ 2.82879405e-03,  6.92966248e-03,  7.89162493e-01],
         [ 1.96276803e-03,  1.10118504e-02,  7.86740709e-01],
         [ 5.45619087e-04,  1.36144114e-02,  7.85241975e-01],
         [ 3.35214028e-04,  1.98778569e-02,  7.81547001e-01],
         [ 5.20588747e-03,  1.84377985e-02,  7.82196197e-01],
         [ 9.58129527e-03,  1.51065844e-02,  7.83613317e-01],
         [ 9.00115199e-03,  2.05432436e-02,  7.80462724e-01],
         [ 7.83056431e-03,  2.41809148e-02,  7.78522335e-01],
         [ 6.54655792e-03,  2.76233732e-02,  7.76690599e-01],
         [ 6.10283236e-03,  3.36162626e-02,  7.73328912e-01],
         [ 1.05053175e-02,  3.08069612e-02,  7.74267333e-01],
         [ 1.49433130e-02,  2.83992800e-02,  7.74203031e-01]

In [34]:
angle = 6.291651496887207
path0, path1 = get_border_paths(angle, paths)
path0, path1

({'points': array([[ 0.00000000e+00,  0.00000000e+00,  7.93512133e-01],
         [ 5.08034674e-03, -5.53042570e-03,  7.96571657e-01],
         [ 5.44256078e-03, -5.52433962e-03,  7.96549058e-01],
         [ 1.07836060e-02, -5.43715246e-03,  7.95820582e-01],
         [ 1.11607408e-02, -5.35097555e-03,  7.95718827e-01],
         [ 1.63223613e-02, -5.30535565e-03,  7.94305587e-01],
         [ 1.67665448e-02, -5.33172011e-03,  7.94196115e-01],
         [ 2.16611999e-02, -5.51115756e-03,  7.91997039e-01],
         [ 2.20629452e-02, -5.75042010e-03,  7.91957316e-01],
         [ 2.67806134e-02, -6.15495309e-03,  7.88894408e-01],
         [ 2.40277034e-02,  1.01518381e-05,  7.86764412e-01],
         [ 2.14978586e-02,  6.38206863e-03,  7.84463260e-01],
         [ 2.60745010e-02,  5.83764540e-03,  7.81471284e-01],
         [ 2.67163914e-02,  5.12086786e-03,  7.81331851e-01],
         [ 3.04990804e-02,  4.32298891e-03,  7.78106674e-01],
         [ 3.16170321e-02,  3.41865502e-03,  7.77387156e-01]

In [38]:
angle = 0
path0, path1 = get_border_paths(angle, paths)
path0, path1

({'points': array([[ 0.00000000e+00,  0.00000000e+00,  7.93512133e-01],
         [ 5.08062815e-03, -5.53074515e-03,  7.96571835e-01],
         [ 5.44267240e-03, -5.53257337e-03,  7.96554003e-01],
         [ 1.07917593e-02, -5.44596642e-03,  7.95824859e-01],
         [ 1.11612337e-02, -5.36785548e-03,  7.95729002e-01],
         [ 1.63384555e-02, -5.32290616e-03,  7.94311985e-01],
         [ 1.67677734e-02, -5.35704095e-03,  7.94211580e-01],
         [ 2.16835600e-02, -5.53705062e-03,  7.92002881e-01],
         [ 2.20651063e-02, -5.78368792e-03,  7.91977581e-01],
         [ 2.68073528e-02, -6.18845799e-03,  7.88897490e-01],
         [ 2.43941457e-02, -2.63006588e-05,  7.86524501e-01],
         [ 2.15282356e-02,  6.34176007e-03,  7.84467483e-01],
         [ 2.60772386e-02,  5.79704793e-03,  7.81495923e-01],
         [ 2.67484578e-02,  5.07450189e-03,  7.81331982e-01],
         [ 3.05028166e-02,  4.27568583e-03,  7.78135755e-01],
         [ 3.16503560e-02,  3.36683388e-03,  7.77383389e-01]

In [41]:
angle = -0.1
path0, path1 = get_border_paths(angle, paths)
path0, path1

AssertionError: 

In [43]:
pnt0, nrm0 = single_path_sample(path0, sq_no, square_no)
pnt1, nrm1 = single_path_sample(path1, sq_no, square_no)


In [44]:
pnt0, nrm0

(tensor([ 0.0052, -0.0057,  0.7967]), tensor([0.0738, 0.5065, 0.8589]))

In [48]:
pnt0, pnt1, (pnt0 + pnt1)/2

(tensor([ 0.0052, -0.0057,  0.7967]),
 tensor([ 0.0052, -0.0057,  0.7967]),
 tensor([ 0.0052, -0.0057,  0.7967]))

In [49]:
import torch.nn.functional as F

In [51]:
F.normalize(nrm0, dim=0)

tensor([0.0738, 0.5066, 0.8590])